# Vacan text preprocessing

In [1]:
import pandas as pd
import bs4
import re
from IPython.display import display

In [2]:
vacancies = pd.read_pickle('data/vacancies.pickle')

In [3]:
vacancies.head()

,id,name,html,url,site,date
3,1413553876,Разработчик Ruby/Ruby on Rails,"<div class=""b-important b-vacancy-info""><table...",http://irkutsk.hh.ru/vacancy/11833695?query=%D...,hh.ru,2014-10-17 13:51:12
4,1413553877,Ведущий программист 1 С/Руководитель направлен...,"<div class=""b-important b-vacancy-info""><table...",http://irkutsk.hh.ru/vacancy/11926858?query=%D...,hh.ru,2014-10-17 13:51:12
5,1413553878,Java-разработчик,"<div class=""b-important b-vacancy-info""><table...",http://irkutsk.hh.ru/vacancy/11968661?query=%D...,hh.ru,2014-10-17 13:51:12
8,1413553881,IOS разработчик,"<div class=""b-important b-vacancy-info""><table...",http://irkutsk.hh.ru/vacancy/11799508?query=%D...,hh.ru,2014-10-17 13:51:12
9,1413553882,Программист PHP (web-разработка; back-end),"<div class=""b-important b-vacancy-info""><table...",http://irkutsk.hh.ru/vacancy/11639762?query=%D...,hh.ru,2014-10-17 13:51:12


In [4]:
vacancies.date.max()

Timestamp('2015-12-06 12:39:17')

In [5]:
soup = bs4.BeautifulSoup(vacancies.iloc[0].html, 'html.parser')

In [6]:
soup.meta

<meta content="RUR" itemprop="salaryCurrency"><meta content="70000" itemprop="baseSalary">от 70 000 руб.</meta></meta>

# Get salary / text

In [7]:
def get_salary(html: str) -> tuple:
    soup = bs4.BeautifulSoup(html, 'html.parser')
    basesalary = None
    salarycurrency = None
    for meta in soup.findAll("meta"):
        meta_name = meta.get('itemprop', '').lower()
        if meta_name == 'salarycurrency':
            salarycurrency = meta.get('content', '').lower()
        if meta_name == 'basesalary':
            basesalary = int(meta.get('content', ''))
    return salarycurrency, basesalary

def get_text(html: str) -> str:
    try:
        x = re.search(r'</table>.*</div>(.*)Тип занятости', html)
        return x.groups()[0].lower()
    except AttributeError:
        return html.lower()

In [8]:
example_html = vacancies.iloc[5].html
display(get_salary(example_html))
display(get_text(example_html))

('rur', 100000)

'компания «эспокада» (один из лидеров рынка домашнего текстиля) предлагает вам поучаствовать в конкурсе на замещение вакансии «программист-разработчик 1с».   что мы можем предложить вам:  работа в стабильной успешной компании просторный офис в 5 минутах ходьбы от метро «авиамоторная» комфортная рабочая обстановка молодой и дружный коллектив сложные, интересные и амбициозные задачи простор для реализации своего интеллектуального и творческого потенциала перспективы карьерного роста оформление в соответствии с тк рф    необходимые требования:  высшее образование (техническое) релевантный опыт работы от 6 лет понимание бизнес-процессов предприятий оптовой торговли (закупки, склад, производство, транспорт, продажи, клиентский сервис, бухгалтерия) способность мыслить нестандартно умение планировать свое время и всегда доводить работу до конца высокий уровень самоорганизации и ответственности за свою работу знания продуктов семейства 1с и практические навыки работы с ними отличное знание вст

In [9]:
vacancies['text'] = vacancies.html.apply(get_text)

In [10]:
salaries = vacancies.html.apply(get_salary)

In [11]:
vacancies['salary'] = salaries.apply(lambda x: x[1])

In [12]:
vacancies['salary_currency'] = salaries.apply(lambda x: x[0])

In [13]:
vacancies['name'] = vacancies.name.apply(lambda x: x.lower())

In [14]:
vacancies.head(2)

,id,name,html,url,site,date,text,salary,salary_currency
3,1413553876,разработчик ruby/ruby on rails,"<div class=""b-important b-vacancy-info""><table...",http://irkutsk.hh.ru/vacancy/11833695?query=%D...,hh.ru,2014-10-17 13:51:12,"в состав распределенной команды, разрабатывающ...",70000,rur
4,1413553877,ведущий программист 1 с/руководитель направлен...,"<div class=""b-important b-vacancy-info""><table...",http://irkutsk.hh.ru/vacancy/11926858?query=%D...,hh.ru,2014-10-17 13:51:12,медицинская сеть клиник приглашает на работу н...,110000,rur


In [15]:
vacancies_prepared = vacancies[['date', 'name', 'text', 'salary', 'salary_currency']]

In [16]:
vacancies_prepared['currency'] = vacancies_prepared.loc[:,'salary_currency']

/usr/local/lib/python3.4/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
del vacancies_prepared['salary_currency']

In [18]:
vacancies_prepared.head()

,date,name,text,salary,currency
3,2014-10-17 13:51:12,разработчик ruby/ruby on rails,"в состав распределенной команды, разрабатывающ...",70000,rur
4,2014-10-17 13:51:12,ведущий программист 1 с/руководитель направлен...,медицинская сеть клиник приглашает на работу н...,110000,rur
5,2014-10-17 13:51:12,java-разработчик,обязанности: участие в проектах компании по р...,40000,rur
8,2014-10-17 13:51:12,ios разработчик,"привет. меня зовут егор, в 2011 году мы создал...",120000,rur
9,2014-10-17 13:51:12,программист php (web-разработка; back-end),в телекоммуникационную компанию требуется прог...,80000,rur


In [19]:
vacancies_prepared.to_pickle('data/vacancies_prepared.pickle')

# Tokenizing

In [62]:
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')
stop_words.extend(['что', 'это', 'так', 'вот', 'быть', 'как', 'в', '—', 'к', 'на'])
stop_words.extend(['разработчик', 'программист', 'developer', 'разработка'])  # add custom stopwords
stop_words = set(stop_words)

In [63]:
import string
punctuation = set(string.punctuation)

In [64]:
from nltk.tokenize import RegexpTokenizer
import nltk
tokenizer = RegexpTokenizer(r'\w+')

In [65]:
def tokenize(text: str) -> list:
    text = text.replace('/', ' ').replace('-', ' ')
    tokens = nltk.word_tokenize(text)
    tokens = [i for i in tokens if i not in stop_words and i not in punctuation]
    tokens = [i.replace("«", "").replace("»", "") for i in tokens]
    return tokens

In [66]:
vacancies_tokens = pd.DataFrame()

In [67]:
vacancies_tokens['name_tokens'] = vacancies_prepared.name.apply(tokenize)

In [68]:
vacancies_tokens['text_tokens'] = vacancies_prepared.text.apply(tokenize)

In [69]:
vacancies_tokens.head()

,name_tokens,text_tokens
3,"[ruby, ruby, on, rails]","[состав, распределенной, команды, разрабатываю..."
4,"[ведущий, 1, руководитель, направления, it]","[медицинская, сеть, клиник, приглашает, работу..."
5,[java],"[обязанности, участие, проектах, компании, раз..."
8,[ios],"[привет, зовут, егор, 2011, году, создали, ком..."
9,"[php, web, back, end]","[телекоммуникационную, компанию, требуется, ph..."


In [70]:
back = vacancies_tokens.copy()

## Stemming

In [71]:
stemmer = nltk.stem.snowball.RussianStemmer(ignore_stopwords=True)

In [72]:
english_letters = set(string.ascii_lowercase)

In [73]:
def stem(tokens: list) -> list:
    return [stemmer.stem(token) if len(token) > 0 and token[0] not in english_letters else token for token in tokens]

In [74]:
vacancies_tokens['name_tokens'] = vacancies_tokens.name_tokens.apply(stem)

In [75]:
vacancies_tokens.head()

,name_tokens,text_tokens
3,"[ruby, ruby, on, rails]","[состав, распределенной, команды, разрабатываю..."
4,"[ведущ, 1, руководител, направлен, it]","[медицинская, сеть, клиник, приглашает, работу..."
5,[java],"[обязанности, участие, проектах, компании, раз..."
8,[ios],"[привет, зовут, егор, 2011, году, создали, ком..."
9,"[php, web, back, end]","[телекоммуникационную, компанию, требуется, ph..."


In [213]:
vacancies_tokens['text_tokens'] = vacancies_tokens.text_tokens.apply(stem)

In [215]:
vacancies_tokens_back = vacancies_tokens.copy()

# Got processed texts

In [79]:
vacancies = vacancies_tokens.join(vacancies_prepared[['date', 'salary', 'currency']])

In [81]:
vacancies.head(20)

,name_tokens,text_tokens,date,salary,currency
3,"[ruby, ruby, on, rails]","[состав, распределенной, команды, разрабатываю...",2014-10-17 13:51:12,70000,rur
4,"[ведущ, 1, руководител, направлен, it]","[медицинская, сеть, клиник, приглашает, работу...",2014-10-17 13:51:12,110000,rur
5,[java],"[обязанности, участие, проектах, компании, раз...",2014-10-17 13:51:12,40000,rur
8,[ios],"[привет, зовут, егор, 2011, году, создали, ком...",2014-10-17 13:51:12,120000,rur
9,"[php, web, back, end]","[телекоммуникационную, компанию, требуется, ph...",2014-10-17 13:51:12,80000,rur
10,[1с],"[компания, эспокада, лидеров, рынка, домашнего...",2014-10-17 13:51:12,100000,rur
11,[],"[обязанности, написание, новых, программ, авто...",2014-10-17 13:51:12,50000,rur
12,"[инженер, машин, обучен, искусствен, интеллект]","[обязанности, эффективная, реализация, алгорит...",2014-10-17 13:51:12,80000,rur
14,"[1с, 8.2, системн, администратор]","[1с, 8.2, системный, администратор, обязанност...",2014-10-17 13:51:12,80000,rur
15,[python],"[обязанности, серверной, части, умного, дома, ...",2014-10-17 13:51:12,50000,rur


In [82]:
vacancies.to_pickle('data/vacancies_stemmed.pickle')

In [96]:
vacancies_back = vacancies.copy()

# TF-Idf features

In [104]:
vacancies = vacancies_back.copy()

In [105]:
vacancies.name_tokens = vacancies.name_tokens.apply(lambda lst: ' '.join(lst))  # create texts
vacancies.text_tokens = vacancies.text_tokens.apply(lambda lst: ' '.join(lst))  # create texts

In [106]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [107]:
vectorizer = TfidfVectorizer(min_df=1)
text_tfidf_matrix = vectorizer.fit_transform(vacancies.name_tokens)

In [110]:
text_tfidf_matrix

<17446x1931 sparse matrix of type '<class 'numpy.float64'>'
	with 40945 stored elements in Compressed Sparse Row format>

In [108]:
vectorizer = TfidfVectorizer(min_df=1)
title_tfidf_matrix = vectorizer.fit_transform(vacancies.text_tokens)

In [112]:
title_tfidf_matrix

<17446x60994 sparse matrix of type '<class 'numpy.float64'>'
	with 2045731 stored elements in Compressed Sparse Row format>

# Very basic linear regression model

see: http://scikit-learn.org/stable/auto_examples/linear_model/plot_ols.html

In [120]:
X = text_tfidf_matrix.todense()
Y = vacancies.salary

In [125]:
from sklearn.preprocessing import Imputer
X = Imputer().fit_transform(X)

In [126]:
import sklearn.linear_model

# Create linear regression object
regr = sklearn.linear_model.LinearRegression()

# Train the model using the training sets
regr.fit(X, Y)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [ ]:
# TODO: Split the data into training/testing sets